In [1]:
import numpy as np
import networkx as nx

from graphqec import RepetitionCode, RotatedSurfaceCode, CssCode, BivariateBicycleCode,commutation_test

The Tanner graph is the representation used here for CSS codes. Identify logical operators using the Tanner graph data structure.

The logic_check is a list of strings, with same structure as the checks.

In [8]:
bbc = BivariateBicycleCode(
    Lx = 6,
    Ly = 6,
    a1 = 3,
    a2 = 1,
    a3 = 2,
    b1 = 3,
    b2 = 1,
    b3 = 2,
)

In [9]:
Hx = bbc.A1+bbc.A2+bbc.A3
Hz = bbc.B1+bbc.B2+bbc.B3

In [20]:
commutation_test(hx,hz)

True

In [13]:
lal = [not hx.dot(hz) for hx in Hx for hz in Hz]

In [19]:
hx = np.hstack((bbc.A,bbc.B))
hz = np.hstack((bbc.B.T, bbc.A.T))

In [16]:
Hx[0]

array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [17]:
Hz[4]

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [9]:
for h in Hx:
    h = list(h)

[0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [3]:
rep = RepetitionCode(
    distance = 5,
    depolarize1_rate = 0.01,
    depolarize2_rate = 0.01
)

In [26]:
rot = RotatedSurfaceCode(
    distance = 5,
    depolarize1_rate = 0.01,
    depolarize2_rate = 0.01
)

0
1
2
3
4
5
6
7
8


In [17]:
rep.get_check_label

<bound method BaseCode.get_check_label of <graphqec.codes.repetition_code.RepetitionCode object at 0x000001FED1909D50>>

In [23]:
for node in graph.nodes():
    print(nx.get_node_attributes(graph,node))

{}
{}
{}
{}
{}
{}
{}
{}
{}
